# **Overview**

This noteboook dedicated to data inspection and data merger. Since our raw dataset is given in years and by cities, for easier modeling, we merge them into one single dataset. 

# **Library and Configuration**

In [1]:
# Module and pathing helper
import sys
sys.path.append('..')

# Ignore warnings
from warnings import filterwarnings

# Core library
import pandas as pd
import numpy as np
import re

# Path library
import os
import glob
from pathlib import Path

# Source code moduls
import importlib
import src.cleaning as cleaning
def r(module=cleaning):
    importlib.reload(module)

r()

# Defaults
filterwarnings('ignore')
np.set_printoptions(legacy='1.25')

# Paths
RAW_PATH = Path('../data/raw')

PROCESS_PATH = Path('../data/process')
MERGE_PATH = PROCESS_PATH/'merge'

# **Data Inspection**

In [2]:
train_admiralty_2009 = pd.read_csv(RAW_PATH/'train/Admiralty/Data_Gabungan_Lainnya_2009.csv')
train_admiralty_2009.head()

,Date,Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,2009-01-01,,,,,,,,
1,2009-01-02,,,,,,,,
2,2009-01-03,,,,,,,,
3,2009-01-04,,,,,,,,
4,2009-01-05,,,,,,,,


In [3]:
train_admiralty_2009.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Date                           365 non-null    object
 1   Highest 30 Min Rainfall (mm)   365 non-null    object
 2   Highest 60 Min Rainfall (mm)   365 non-null    object
 3   Highest 120 Min Rainfall (mm)  365 non-null    object
 4   Mean Temperature (°C)          365 non-null    object
 5   Maximum Temperature (°C)       365 non-null    object
 6   Minimum Temperature (°C)       365 non-null    object
 7   Mean Wind Speed (km/h)         365 non-null    object
 8   Max Wind Speed (km/h)          365 non-null    object
dtypes: object(9)
memory usage: 25.8+ KB


In [4]:
train_admiralty_2009.iloc[1,1]

'\x97'

There is invalid encoding that counts as a valid value in our dataset, but first of all, lets just merge each city to a single dataset containing every year data.

In [5]:
def check_columns_consistency(root_dir):
    columns_map = {}

    csv_files = glob.glob(
        os.path.join(root_dir, "**", "*.csv"),
        recursive=True
    )

    if not csv_files:
        raise ValueError("No CSV files found")

    for f in csv_files:
        cols = pd.read_csv(f, nrows=0).columns

        normalized_cols = tuple(sorted(
            col.strip() for col in cols
        ))

        columns_map.setdefault(normalized_cols, []).append(f)

    return columns_map

In [ ]:
train_columns = check_columns_consistency(RAW_PATH / "train")
print(f"Found {len(train_columns)} unique column structures\n")

for i, (cols, files) in enumerate(train_columns.items(), 1):
    print(f"[Structure {i}]")
    print(f"Columns ({len(cols)}): {cols}")
    print(f"Used by {len(files)} files")
    print("-" * 90)

Found 2 unique column structures

[Structure 1]
Columns (9): ('Date', 'Highest 120 Min Rainfall (mm)', 'Highest 30 Min Rainfall (mm)', 'Highest 60 Min Rainfall (mm)', 'Max Wind Speed (km/h)', 'Maximum Temperature (°C)', 'Mean Temperature (°C)', 'Mean Wind Speed (km/h)', 'Minimum Temperature (°C)')
Used by 1020 files
------------------------------------------------------------------------------------------
[Structure 2]
Columns (9): ('Date', 'Highest 120 min Rainfall (mm)', 'Highest 30 min Rainfall (mm)', 'Highest 60 min Rainfall (mm)', 'Max Wind Speed (km/h)', 'Maximum Temperature (°C)', 'Mean Temperature (°C)', 'Mean Wind Speed (km/h)', 'Minimum Temperature (°C)')
Used by 132 files
------------------------------------------------------------------------------------------


In [8]:
test_columns = check_columns_consistency(RAW_PATH/'test')
print(f"Found {len(test_columns)} unique column structures\n")

for i, (cols, files) in enumerate(test_columns.items(), 1):
    print(f"[Structure {i}]")
    print(f"Columns ({len(cols)}): {cols}")
    print(f"Used by {len(files)} files")
    print("-" * 90)

Found 1 unique column structures

[Structure 1]
Columns (9): ('Date', 'Highest 120 min Rainfall (mm)', 'Highest 30 min Rainfall (mm)', 'Highest 60 min Rainfall (mm)', 'Max Wind Speed (km/h)', 'Maximum Temperature (°C)', 'Mean Temperature (°C)', 'Mean Wind Speed (km/h)', 'Minimum Temperature (°C)')
Used by 88 files
------------------------------------------------------------------------------------------


# **Merge Years**

In [9]:
cleaning.merge_each_city(
    input_root=RAW_PATH/'train',
    output_dir=MERGE_PATH/'train',
)

[INFO] Processing 44 city folders...

[OK] Admiralty              |   15 files |     5,448 rows |  9 cols
[OK] Ang_Mo_Kio             |   15 files |     5,235 rows |  9 cols
[OK] Bukit_Panjang          |   44 files |    16,013 rows |  9 cols
[OK] Bukit_Timah            |   15 files |     5,327 rows |  9 cols
[OK] Buona_Vista            |   15 files |     5,388 rows |  9 cols
[OK] Changi                 |   44 files |    16,071 rows |  9 cols
[OK] Choa_Chu_Kang_Central  |   15 files |     5,144 rows |  9 cols
[OK] Choa_Chu_Kang_South    |   13 files |     4,627 rows |  9 cols
[OK] Clementi               |   44 files |    16,071 rows |  9 cols
[OK] East_Coast_Parkway     |   15 files |     5,327 rows |  9 cols
[OK] Jurong_Island          |   13 files |     4,536 rows |  9 cols
[OK] Jurong_Pier            |   44 files |    16,071 rows |  9 cols
[OK] Jurong_West            |   44 files |    15,887 rows |  9 cols
[OK] Kent_Ridge             |   15 files |     5,419 rows |  9 cols
[OK] Kranj

In [10]:
cleaning.merge_each_city(RAW_PATH/'test', MERGE_PATH/'test', cleaning.clean_column_names)

[INFO] Processing 44 city folders...

[OK] Admiralty              |    2 files |       517 rows |  9 cols
[OK] Ang_Mo_Kio             |    2 files |       517 rows |  9 cols
[OK] Bukit_Panjang          |    2 files |       517 rows |  9 cols
[OK] Bukit_Timah            |    2 files |       517 rows |  9 cols
[OK] Buona_Vista            |    2 files |       517 rows |  9 cols
[OK] Changi                 |    2 files |       517 rows |  9 cols
[OK] Choa_Chu_Kang_Central  |    2 files |       517 rows |  9 cols
[OK] Choa_Chu_Kang_South    |    2 files |       517 rows |  9 cols
[OK] Clementi               |    2 files |       517 rows |  9 cols
[OK] East_Coast_Parkway     |    2 files |       517 rows |  9 cols
[OK] Jurong_Island          |    2 files |       517 rows |  9 cols
[OK] Jurong_Pier            |    2 files |       517 rows |  9 cols
[OK] Jurong_West            |    2 files |       517 rows |  9 cols
[OK] Kent_Ridge             |    2 files |       517 rows |  9 cols
[OK] Kranj

In [11]:
admiralty_merged = pd.read_csv(MERGE_PATH/'train/Admiralty.csv')
admiralty_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5448 entries, 0 to 5447
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   date                         5448 non-null   object
 1   highest_30_min_rainfall_mm   5153 non-null   object
 2   highest_60_min_rainfall_mm   5153 non-null   object
 3   highest_120_min_rainfall_mm  5153 non-null   object
 4   mean_temperature_c           5424 non-null   object
 5   maximum_temperature_c        5424 non-null   object
 6   minimum_temperature_c        5424 non-null   object
 7   mean_wind_speed_kmh          5394 non-null   object
 8   max_wind_speed_kmh           5395 non-null   object
dtypes: object(9)
memory usage: 383.2+ KB


# **Merge Cities**

In [15]:
train = cleaning.merge_all_cities(MERGE_PATH/'train', output_dir=PROCESS_PATH/'train.csv')
train = cleaning.clean_column_names(train)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416121 entries, 0 to 416120
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   date                         416121 non-null  object 
 1   highest_30_min_rainfall_mm   403151 non-null  object 
 2   highest_60_min_rainfall_mm   403148 non-null  object 
 3   highest_120_min_rainfall_mm  403151 non-null  object 
 4   mean_temperature_c           379428 non-null  object 
 5   maximum_temperature_c        380303 non-null  object 
 6   minimum_temperature_c        380263 non-null  object 
 7   mean_wind_speed_kmh          379149 non-null  object 
 8   max_wind_speed_kmh           379790 non-null  object 
 9   location                     416121 non-null  object 
 10  daily_rainfall_total_mm      0 non-null       float64
dtypes: float64(1), object(10)
memory usage: 34.9+ MB


In [16]:
test = cleaning.merge_all_cities(MERGE_PATH/'test', output_dir=PROCESS_PATH/'test.csv')
test = cleaning.clean_column_names(test)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22748 entries, 0 to 22747
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date                         22748 non-null  object 
 1   highest_30_min_rainfall_mm   21911 non-null  float64
 2   highest_60_min_rainfall_mm   21953 non-null  float64
 3   highest_120_min_rainfall_mm  21953 non-null  float64
 4   mean_temperature_c           8999 non-null   float64
 5   maximum_temperature_c        8976 non-null   float64
 6   minimum_temperature_c        8976 non-null   float64
 7   mean_wind_speed_kmh          8033 non-null   float64
 8   max_wind_speed_kmh           8033 non-null   float64
 9   location                     22748 non-null  object 
 10  daily_rainfall_total_mm      0 non-null      float64
dtypes: float64(9), object(2)
memory usage: 1.9+ MB
